In [1]:
# network/comp. bio. packages
import networkx as nx
import cobra
from cobra.io import read_sbml_model
import netwulf
from netwulf import visualize

# helper packages
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import random

%matplotlib inline

In [7]:
# iterate over model.reactions and model.metabolites to construct bipartite directed network
def makeNetworkFromSBML(model):
    # directed grah
    G = nx.DiGraph()
    
    # metabolite nodes and reaction nodes
    nodes_m = {}
    nodes_r = {}

    # iterate through metabolites in model and add to nodes_m dict
    for metabolite in model.metabolites:
        nodes_m[metabolite.id] = metabolite.name
        
    # iterate through reactions in model and add to nodes_r dict
    for reaction in model.reactions:
        nodes_r[reaction.id] = reaction.name
    
    # add nodes from met/rxn dicts with bipartite attribute
    # add nodes with metabolite formula/enzyme name as attribute for id
    G.add_nodes_from([(m_id, {'name': name}) for (m_id, name) in nodes_m.items()], bipartite=0)
            # same as...
            # for m_id, name in nodes_m.items():
                # G.add_node(m_id, name = name, biartite = 0)
    
    G.add_nodes_from([(r_id, {'name': name}) for (r_id, name) in nodes_r.items()], bipartite=1)
    
    edges = []
    for reaction in model.reactions:
        # get products and reactants
        products  = reaction.products
        reactants = reaction.reactants
        enzyme = reaction.name
        rid = reaction.id
        
        for p in products:
            edges.append((rid, p.id))
            if reaction.reversibility:
                edges.append((p.id, rid))
                
        for r in reactants:
            edges.append((r.id, rid))
            if reaction.reversibility:
                edges.append((rid, r.id))
            
        
    G.add_edges_from(edges)
    return G


In [8]:
# create list of metabolic networks, one for each genome
networks = {}
models = {}
sbml_dir = 'sbml_files'
for sbml_file in os.listdir(sbml_dir):
    # print(sbml_file)
    
    # ignore JCVI for now...
    if sbml_file != 'JCVI.sbml':
        model = read_sbml_model(os.path.join(sbml_dir, sbml_file))
        models[sbml_file] = model
        networks[sbml_file] = makeNetworkFromSBML(model)

In [42]:
enzymes = set()

for model in models.values():
    enzymes.update(set(r.id for r in model.reactions))

In [43]:
metabolites = set()

for model in models.values():
    metabolites.update(set(m.id for m in model.metabolites))

In [44]:
unique_reactions = []

for e in enzymes:
    update = 0
    for f,n in networks.items(): 
        if e in n.nodes:
            update += 1
    
    if update < len(networks.items()):
        # print("not all species contain: ", e )
        unique_reactions.append(e)


In [45]:
unique_metabolites = []

for m in metabolites:
    update = 0
    for f,n in networks.items(): 
        if m in n.nodes:
            update += 1
    
    if update < len(networks.items()):
        # print("not all species contain: ", m )
        unique_metabolites.append(e)

not all species contain:  cpd00572_e
not all species contain:  cpd00312_c


### Create JCVI model 

In [50]:
JCVI_model = read_sbml_model('JCVI.sbml')

In [51]:
G = makeNetworkFromSBML(JCVI_model)

In [54]:
degree_list = sorted([d for n, d in G.degree()], reverse = True)
# print(degree_list)
# print(len(degree_list))

print(len(G.nodes()))

1057


In [58]:
len(JCVI_model.metabolites)


567

In [53]:
missing_enz = []

j = 1
for e in enzymes:
    if e not in G.nodes():
        missing_enz.append(e)
        print(j, ": JCVI doesn't contain the enzyme: ", e)
        j += 1

        
print(len(missing_enz))

1 : JCVI doesn't contain the enzyme:  rxn09536_c
2 : JCVI doesn't contain the enzyme:  rxn20057_c
3 : JCVI doesn't contain the enzyme:  rxn12863_c
4 : JCVI doesn't contain the enzyme:  rxn08922_c
5 : JCVI doesn't contain the enzyme:  rxn05573_c
6 : JCVI doesn't contain the enzyme:  rxn29990_c
7 : JCVI doesn't contain the enzyme:  rxn10770_c
8 : JCVI doesn't contain the enzyme:  rxn05169_c
9 : JCVI doesn't contain the enzyme:  rxn45359_c
10 : JCVI doesn't contain the enzyme:  rxn05241_c
11 : JCVI doesn't contain the enzyme:  rxn08241_c
12 : JCVI doesn't contain the enzyme:  rxn47850_c
13 : JCVI doesn't contain the enzyme:  rxn16009_c
14 : JCVI doesn't contain the enzyme:  rxn34108_c
15 : JCVI doesn't contain the enzyme:  rxn23650_c
16 : JCVI doesn't contain the enzyme:  rxn48239_c
17 : JCVI doesn't contain the enzyme:  rxn00124_c
18 : JCVI doesn't contain the enzyme:  rxn02835_c
19 : JCVI doesn't contain the enzyme:  rxn25073_c
20 : JCVI doesn't contain the enzyme:  rxn02473_c
21 : JCVI

### Weighted Projection on Enzymes
Function to create a weighted projection of G onto its enzyme node sets. G reaction nodes must have the attribute $bipartite == 1$

In [49]:
mets, rxns = nx.bipartite.sets(G)

# Projection of JCVI onto reaction nodes
# edge weight is the ratio between actual shared neighbors and max possible shared neighbors
# (if ratio == False) edge weight is just the number of shared neigbors
G_rxns = nx.bipartite.weighted_projected_graph(G, rxns, ratio = False)

print(G_rxns)
print(G)

DiGraph with 306 nodes and 18222 edges
DiGraph with 609 nodes and 2302 edges


(None, None)

In [25]:
edge_weights = []
for i in G_rxns.edges():
    edge_weights.append(G_rxns.edges[i]['weight'])

sorted(edge_weights, reverse = True)
print(len(edge_weights))

18222


In [47]:
# data = visualize(G_rxns)